currently, I'm working on a competition for Protein folding, and this my first time using the library. So, I though it would be a good idea to do gene expression profiles since I'm somewhat knowledgeable about the subject now. 

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('gene_expression_data.csv')

# Cleaning
X = df.drop('target', axis=1).values
y = df['target'].values


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Conversion to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


In [ ]:
import torch.nn as nn
from ndlinear import NdLinear

class GeneExpressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GeneExpressionModel, self).__init__()
        self.layer1 = NdLinear(input_dim, 128)
        self.layer2 = NdLinear(128, 64)
        self.layer3 = NdLinear(64, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return self.softmax(x)


In [ ]:
#Training
input_dim = X_train.shape[1]
output_dim = len(torch.unique(y_train))

model = GeneExpressionModel(input_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
#Testing
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / y_test.size(0)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
